# Pràctiques de Nous Usos de la Informàtica

**Nom de les persones del grup:** Jordi Olivares Provencio & Christian José Soler

## Pràctica 3: Naive Bayes i Classificació

<b>Què s’ha de fer?</b><br>
Volem classificar notícies corresponents al diari New York Times segons de quin tòpic parlin principalment. A partir  de totes les notíces que tenim guardades en un fitxer .csv, crearem un vector de característiques que ens descrigui  cada notícia. Finalment desenvoluparem un classificador probabilístic del tipus Naive Bayes que ens permeti identificar a quin tòpic pertany una notícia donada segons les característiques triades.<br>

<b>Quina és la idea del sistema de classificació que s’ha de desenvolupar?</b><br>
L'objectiu del classificador és, donat un vector de característiques que descriuen els objectes que es volen classificar, indicar a quina categoria o classe pertanyen d'entre un conjunt predeterminat. El procés de classificació consta de dues parts:

+ el procés d'aprenentatge 
+ el procés d'explotació o testeig. 

El procés d'aprenentatge rep exemples de parelles $(x,y)$ on $x$ són les característiques, usualment nombres reals, i $y$ és la categoria a la que pertanyen. Aquest conjunt se'l coneix com a conjunt d'entrenament i ens servirà per trobar una funció $y=h(x)$ que donada una $x$ m'indiqui quina és la $y$. Per altra banda el procés de testeig aplica la funció $h(x)$ apresa a l'entrenament a una nova descripció per veure quina categoria li correspon.</br>

<b>Classificació i llenguatge natural</b><br>
La descripció dels exemples en característiques és el punt més crític de tot sistema d'aprenentatge automàtic. Una de les representacions més simples per tal de descriure un text és la representació bag-of-words. Aquesta representació converteix un text en un vector de $N$ paraules. Consisteix en seleccionar un conjunt d'$N$ paraules i per cada paraula contar quants cops apareix en el text. Una versió alternativa d'aquest procés pot ser simplement indicar si apareix o no en el text.

<b>Dades del New York Times</b>
En aquest exemple, el nostre objectiu és classificar automàticament les notícies d'acord al seu titular en vint temes o tòpics. Les dades que disposem són cadascú dels articles de primera plana del New York Times entre 1996 i 2006, classificats segons Policy Agendas (http://www.policyagendas.org ). Aquesta recollida de dades l'ha compilat n'Amber E. Boydstun.

Concretament, els tòpics són els següents

<table border="1">
<tr>
<td>
1 
<td>
Macroeconomics
<tr>
<td>
2 
<td>
Civil Rights, Minority Issues, and Civil Liberties 
<tr>
<td>
3
<td>
Health
<tr>
<td>
4 
<td>Agriculture
<tr>
<td>
5 
<td>Labor, Employment, and Immigration
<tr>
<td>
6 
<td> Education
<tr>
<td>
7
<td>Environment
<tr>
<td>
8
<td>Energy
<tr>
<td>
10 
<td>Transportation
<tr>
<td>
12 
<td>Law, Crime, and Family Issues
<tr>
<td>
13 
<td>Social Welfare
<tr>
<td>
14 
<td>Community Development and Housing Issues
<tr>
<td>
15 
<td>Banking, Finance, and Domestic Commerce
<tr>
<td>
16 
<td>Defense
<tr>
<td>
17 
<td>Space, Science, Technology and Communications
<tr>
<td>
18 
<td>Foreign Trade
<tr>
<td>
19 
<td>International Affairs and Foreign Aid
<tr>
<td>
20 
<td>Government Operations
<tr>
<td>
21 
<td>Public Lands and Water Management
<tr>
<td>
24 
<td>State and Local Government Administration
<tr>
<td>
26 
<td>Weather and Natural Disasters
<tr>
<td>
27 
<td>Fires
<tr>
<td>
28 
<td>Arts and Entertainment
<tr>
<td>
29 
<td>Sports and Recreation
<tr>
<td>
30 
<td>Death Notices
<tr>
<td>
31 
<td>Churches and Religion
<tr>
<td>
99 
<td>Other, Miscellaneous, and Human Interest


<b>Exemple.</b>
Imaginem que tenim 4 notícies que pertanyen només a dues categories ``y={'8. Energy', '2. Civil Rights, Minority Issues, and Civil Liberties'}``, podríem seleccionar les següents paraules per tal de distingir les dues categories ``x={'nuclear', 'verd', 'ecologia', 'independència', 'autonomia', 'referèndum'}``.

<table border="1">
<tr>
<td></td>
<td>nuclear</td>
<td>verd</td>
<td>ecologia</td>
<td>independència</td>
<td>autonomia</td>
<td>referèndum</td>
</tr>
<tr>
<td>news 1('2. Civil Rights...')</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>1</td>
<td>2</td>
<td>3</td>
</tr>
<tr>
<td>news 2('2. Civil Rights...')</td>
<td>0</td>
<td>1</td>
<td>0</td>
<td>1</td>
<td>2</td>
<td>3</td>
</tr>
<tr>
<td>news 3('8. Energy')</td>
<td>1</td>
<td>3</td>
<td>1</td>
<td>0</td>
<td>1</td>
<td>0</td>
</tr>
<tr>
<td>news 4('8. Energy')</td>
<td>2</td>
<td>1</td>
<td>2</td>
<td>0</td>
<td>0</td>
<td>0</td>
</tr>
</table>

<br>
amb aquesta representació la notícia 2, corresponent a la categoria ``'2. Civil Rights...'``, quedaria representat pel vector numèric ``(0,1,0,1,2,3)``. Si fem servir la representació alternativa (booleana) tindríem ``(0,1,0,1,1,1)`` que indica la presència de les paraules. Si la descripció és adient s'espera que les categories es puguin distingir entre elles amb facilitat.
<br><br>
<b>El classificador Naïve Bayes.</b><br>
Un cop tenim una representació necessitem un procés d'aprenentatge que ens permeti passar de la descripció a una categoria. En aquesta pràctica farem servir el classificador Naïve Bayes. Aquest classificador forma part de la família de classificadors probabilístics. La sortida d'un classificador probabilístic és un valor de probabilitat donat un exemple per cadascuna de les categories. La decisió final correspon a la categoria amb més probabilitat. Per exemple, amb la descripció anterior esperem que la sortida sigui de l'estil,<br>

$$p( y = 'Civil\_Rights' \; | \; x = (0,1,0,1,1,1)) = 0.6$$
$$p( y = 'Energy' \; | \; x = (0,1,0,1,1,1)) = 0.4$$

<br>
Els classificadors probabilistics Bayesians es basen en el teorema de Bayes per realitzar els càlculs per trobar la probabilitat condicionada: <br>

$$ p(x,y) = p(x|y)p(y) = p(y|x)p(x)$$
<br>
d'on podem extreure que: <br>
$$ p(y,x) = \frac{p(x|y)p(y)}{p(x)}$$
<br>


En molts casos p(y) i p(x) són desconeguts i es consideren equiprobables. Per tant, la
decisió es simplifica a:
<br>
$$ p(y|x) = c · p(x|y)$$

<br>
Les deduccions fins a aquest punt són vàlides per la majoria de classificadors Bayesians. Naïve Bayes es distingeix de la resta perquè imposa una condició encara més restrictiva. Considerem $x=(x_1,x_2,x_3,...,x_N)$ un conjunt d'$N$ variables aleatòries. Naïve Bayes assumeix que totes elles són independents entre elles i per tant podem escriure:
<br>
$$p(x_1,x_2,...,x_N | y) = p(x_1|y)p(x_2|y)...p(x_N|y)$$

<br>
Per tant en el nostre cas es pot veure com:
<br>
$$p(y='Civil\_Rights' \; |\;x=(0,1,0,1,1,1)) = p(x_1=0|\;'Civil\_Rights' \; )p(x_2=1|\;'Civil\_Rights' \; )...p(x_6=1|\;'Civil\_Rights' \; )$$

<br>
Podem interpretar l'anterior equació de la següent forma: La probabilitat de que el document descrit pel vector de característiques (0,1,0,1,1,1) sigui de la classe "Civil Rights" és proporcional al producte de la probabilitat que la primera paraula del vector (nuclear) no aparegui en les notícies sobre "Civil Rights"  per la probabilitat que la segona paraula sí que hi aparegui, etc.

<br>
<b>Estimant les probabilitats marginals condicionades</b>
L'últim pas que ens queda és trobar el valor de les probabilitats condicionades. Farem servir la representació de 0's i 1's indicant que la paraula no apareix (0) o sí apareix (1) a la notícia. Per trobar el valor de la probabilitat condicionada farem servir una aproximació freqüentista a la probabilitat. Això vol dir que calcularem la freqüència d'aparició de cada paraula per a cada categoria. Aquest càlcul es fa dividint el nombre de notícies de la categoria en que apareix la paraula pel nombre total de notícies d'aquella categoria. En l'exemple anterior, 
$p('verd'=1 |y='Civil\_Rights')=1/2 $, mentre que  $p('verd' =1 |y='Energy')=2/2 $

En general:
<br>
$$p(x = 1 | y = C)= \frac{A}{B} $$
<br>
on A és el número de notícies de la categoria C on hi apareix la paraula 'x' i B és el número total de notícies de la categoria C.


<b>Punts dèbils: </b><br>

<b> * El problema de la probabilitat 0</b>
Si us hi fixeu bé, en l'anterior exemple la probabilitat <b>p('verd'= 0 | y='Energy')</b> és 0 !! Això vol dir, que si en la notícia no hi apareix la paraula 'verd' no pot ser classificada com a categoria 'Energy'!! No sembla raonable que s'assigni o no en aquesta categoria segons si en la notícia hi apareix o no una única paraula. Per tant, el que s'acostuma a fer és donar una baixa probabilitat en comptes de zero. Una de les possibles solucions es fer servir la correcció de Laplace. Seguint l'exemple anterior la correcció de Laplace és

<br>
$$p(x=1 | y = 'C' ) = \frac{A+1}{B+M}$$ 
on M és el nombre de catergories

<b> * El problema de com escollir el vector de característiques</b>
L'elecció de les paraules que formen el vector de característiques és un pas crític. En funció de com de bona sigui aquesta descripció, millor funcionarà el sistema. Tot i que us deixem a vosaltres la política de creació del vector de característiques us donem una d'exemple. Per saber quines paraules fer servir podeu seleccionar de totes les paraules de totes les notícies aquelles que apareixen entre en un 10 i un 50 percent del total (sense tenir en compte la categoria). Podeu experimentar variant aquests valors.

<b> * El problema del "underflow"</b>
 La funció que hem de calcular en el Naive Bayes és un producte. El nombre de caractéristiques del vector és el nombre de termes del producte. Aquests nombres són iguals o menors a 1, si els multipliquem tots entre ells el resultat serà massa petit per a representar-lo en un nombre de punt flotant i el càlcul acabarà sent reduït a zero. Per solucionar aquest problema en comptes d'operar fent multiplicacions, se sol passar a l'escala logarítmica i allà operar fent servir sumes en comptes de multiplicacions
 

In [ ]:
%reset -f
#load data
import pandas as pd
import numpy as np
data=pd.read_csv('Boydstun_NYT_FrontPage_Dataset_1996-2006_0.csv',index_col="Article_ID")
data.head()

<b>Exercici 1 </b> Escriure una funció <b>count_news(dataframe)</b> que retorni el nombre total de noticies. 

In [ ]:
#Retorna el número total de notícies. 
def count_news(df):
    return len(df)
print count_news(data)

<b>Exercici 2:</b> Escriure una funció <b>count_topic_news(dataframe)</b> que compti quantes notícies hi ha per cadascun dels tópics. Agaferem el camp "Topic_2digit" com a identificador de tópic.

In [ ]:
#Retorna una Series que compte el número de notícies per a cadascun dels tópics
def count_topic_news(df):
    return df.groupby('Topic_2digit').count()['Date']

count_topic_news(data)

<b>Exercici 3:</b> Escriure una funció <b>count_words(dataframe)</b> que retorni un diccionari amb totes les paraules que hi hagi en totes les notícies, indicant per cada paraula quantes ocurrències en total hi ha i en quantes notícies surt.
<br>Possible format sortida: {word :  {n_ocur: valor;n_news: valor}}

In [ ]:
# Aquesta funció ha de construir un diccionari que contingui totes les paraules que s'han trobat indicant 
# el total de cops que ha aparegut i el nombre de notícies on apareix
def count_words(df):        
    word_dicc={}       
    for c in df['Title']:
        tmp = set()
        for word in c.lower().split():
            try:
                word_dicc[word]['n_occur'] += 1
                tmp.add(word)
            except:
                word_dicc[word] = {'n_occur' : 1, 'n_news' : 1}
        for word in tmp:
            word_dicc[word]['n_news'] += 1
    return word_dicc


dicc_text = count_words(data)

<b>Exercici 4:</b> escriure una funció <b>count_words_topic(dataframe,dicc_text)</b> que retorna un diccionari que conté el nombre de cops que ha aparegut  cada paraula i el número de notícies on  ha aparegut. Aquesta informació ha de ser dividida pels diferents tópics de noticies.
<br>Possible format sortida: {Topic :  {word :  {n_ocur: valor;n_news: valor} } } 

In [ ]:
#Compta la freqüència de les paraules per a un tòpic determinat
def count_words_topic(df):
    topic_dicc = {topic : {} for topic in df["Topic_2digit"].unique()}
    for _, article in df.iterrows():
        tmp = set()
        for word in article.loc["Title"].lower().split():
            try:
                topic_dicc[article.loc["Topic_2digit"]][word]['n_occur'] += 1
                if word not in tmp:
                    topic_dicc[article.loc["Topic_2digit"]][word]["n_news"] += 1
            except:
                topic_dicc[article.loc["Topic_2digit"]][word] = {"n_occur" : 1, "n_news" : 1}
            tmp.add(word)
    return topic_dicc

words_topics=count_words_topic(data)

<b>Exercici 5:</b> Calcular amb la funció <b>topNword(df,words_topics,N)</b> quines son les N paraules més representatives (les que apareixen amb més freqüència) de cadascun dels tòpics. Retorneu un diccionari amb els següent format: {1: llista_top_words_topic_1; 2: llista_top_words_topic_2;...}
<br>Teniu en compte que també haureu de filtrar aquelles paraules que apareixen en la majoria de notícies, així com també, les que únicament apareixen en un conjunt molt petit de notícies.


In [ ]:
#Calcula les N paruales més representativa de cada tòpic . La sortida ha de 
# ser un diccionari on tenim tantes entrades com tòpics
# el valors de les entrades ha de ser una llista amb les paraules seleccionades.
from heapq import nlargest
def topNwords(df,words_topics,N):
    top_words = {topic : {} for topic in words_topics}
    for topic in words_topics:
        words = words_topics[topic]
        top_words[topic] = nlargest(N, words, key=lambda x : words[x]['n_occur'])
    return top_words

<b>Exercici 6</b>: Creeu el vector de característiques necessari per a fer l’entrenament del Naïve Bayes (funció <b>create_features()</b>).

In [ ]:
# Crea el vector de característiques necessari per a l'entrenament del classificador Naive Bayes
# selected_words: ha de ser el diccionari que retorna topNWords.
# train_data : conté totes les notícies del conjunt d'entrenament
# Retorna un diccionari que conté un np.array per a cadascuna de les notícies amb el vector de característiques corresponent.

from itertools import chain

def create_features(train_data, topicWords):
    dict_feat_vector = {news : {} for news in train_data.index}
    for index, article in train_data.iterrows():
        vecFeature = np.array([0] * len(topicWords))
        a = set(article.loc["Title"].lower().split())
        for i in xrange(len(vecFeature)):
            vecFeature[i] = 1 if topicWords[i] in a else 0
        dict_feat_vector[index] = vecFeature
        
    return dict_feat_vector

N = 20 # Aquest parametre el podem canviar i fer proves per avaluar quin és el millor valor. 
words_topics=count_words_topic(data)
top_words=topNwords(data,words_topics,N)
topic_words = list(chain.from_iterable(top_words.values()))
dict_feat_vector = create_features(data, topic_words)

<b>Exercici 7</b>: Implementeu la funció d'aprenentatge del classificador Naïve Bayes (funció <b>naive_bayes_learn()</b>).  La funció ha de mostrar per pantalla el resultat obtingut 
<br>
<b> * L'error d'entrenament</b>
L'error d'entrenament es troba calculant el percentatge d'errors que s'obtenen quan es fa el testeig amb les mateixes dades utilizades per fer entrenament (aprenentatge). Aquest error es un valor molt optimista de com funcionarà el clasificador i mai s'ha de prendre com a mesura per comparar clasificadors.
<br>

In [ ]:
# CHRISTIAN: Per totes les notícies, calcular la probabilitat que pertànyi a cada tòpic 
    # i tornar el tòpic més probable per cada notícia

#Mètode que implementa el clasificador Naive_Bayes.Ha de mostrar el resultat obtingut per pantalla

def naive_bayes(df, dict_feat):
    from math import log
    guess = {}
    count_top = count_topic_news(df)
    M = len(count_top)
    for news in dict_feat:
        probs = {}
        vec = dict_feat[news]
        for topic in words_topics:
            probs[topic] = 0
            w = words_topics[topic]
            c = count_top[topic]
            for i, word in enumerate(topic_words):
                try:
                    A = w[word]["n_news"]
                    B = c
                except:
                    A = 1
                    B = c
                probs[topic] += log(A + 1) - log(B + M) if vec[i] else log(B + M - A - 1) - log(B + M)
        guess[news] = max(probs.keys(), key=lambda x: probs[x])
    return guess
## EXEMPLE SORTIDA (LES XIFRES SÓN INVENTADES!):
#Tòpic 1 encerts: 24 / Notícies: 34 / 70.59 %
#Tòpic 2  encerts: 21 / Notícies: 26 / 80.77 %
#Tòpic 3  encerts: 29 / Notícies: 32 / 90.62 %
#Tòpic 4  encerts: 26 / Notícies: 29 / 89.66 %
#
#Error naive bayes: 17.36 %

result = naive_bayes(data, dict_feat_vector)
for key in result:
    print result[key]
    break

<b>Exercici 8: </b> Indiqueu l'error de generalització fent servir *n-fold validation* (funció <b>n_fold ()</b> )

**Aproximació a l'error de generalització fent servir  *n-fold validation* **.
Una bona forma de veure com funcionaria el nostre classificador davant de dades sobre les quals no s'ha entrenat és fer servir l'estratègia n-fold. Aquesta estratègia testeja el classificador amb una partició de les dades d'entrenament i fa l'entrenament sobre la resta de dades que hem exclòs. Aquest procés d'exclusió es repeteix per cadascún dels *folds* de les dades d'entrenament. El nombre de *folds* determina quantes particions hem de fer, i per tant, les dades que hi ha en el conjut de test. Per exemple, en un *5-fold* validation, es fan 5 particions, les dades de test són un cinquè de les dades i l'entrenament es fa amb els quatre cinquens restants. El percentatge d'errors fent servir aquesta estratègia permet comparar classificadors.

In [ ]:
#Mètode per avaluar el classificador mitjançant la tècnica n-fold validation. n és el nombre de folds  
#Ha de mostrar per pantalla el resultat obtingut 
def n_fold(df,feature_vector,n):
     return 0
    
## EXEMPLE SORTIDA (LES XIFRES SÓN INVENTADES!):
#Tòpic 1 encerts: 24 / Notícies: 34 / 70.59 %
#Tòpic 2  encerts: 21 / Notícies: 26 / 80.77 %
#Tòpic 3  encerts: 29 / Notícies: 32 / 90.62 %
#Tòpic 4  encerts: 26 / Notícies: 29 / 89.66 %
#

#Error naive bayes: 17.36 %

<b>Resultat Final</b>

Definició de la funció principal. Modifiqueu la funció per tal que s'ajusti a les vostres funcions.

In [ ]:
# Main. Es criden a totes les funcions per a la correcte execució del programa.   
def main():

    import pandas as pd
    data=pd.read_csv('./files/Boydstun_NYT_FrontPage_Dataset_1996-2006_0.csv')
    N = 20 # Aquest parametre el podem canviar i fer proves per avaluar quin és el millor valor. 
    words_topics=count_words_topic(data,dicc_text)
    top_words=topNwords(data,words_topics,N)
    feature_vectors = create_features(data,top_words) 
    naive_bayes(data,feature_vectors) #fins aquí error d'entrenament. Fem servir totes les dades.
    
    
    n_fold(data, feature_vectors)

In [ ]:
main()